## EDA

In [ ]:
import json

In [ ]:
train_filename = r'../dataset_coco/annotations/coco_annotations_train.json'
val_filename = r'../dataset_coco/annotations/coco_annotations_test.json'
test_filename = r'../dataset_coco/annotations/coco_annotations_val.json'


In [ ]:
import json

def count_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        return len(data['images'])


def count_category_data(filename):
    category_count = {
        "is_up": {"false": 0, "true": 0},
        "color": {"White": 0, "Yellow": 0, "No color": 0},
        "count": {"undefined": 0, "K1": 0, "K2_1": 0, "K2_2": 0, "K3": 0, "K4": 0}
    }

    with open(filename, 'r') as file:
        data = json.load(file)

        for annotation in data['annotations']:
            is_up = annotation["is_up"]
            color = annotation["color"]
            count = annotation["count"]

            category_count["is_up"][is_up] += 1
            category_count["color"][color] += 1
            category_count["count"][count] += 1

        return category_count

# 1. Количество данных в каждом файле

In [ ]:
train_count = count_data(train_filename)
val_count = count_data(val_filename)
test_count = count_data(test_filename)

print(f'Количество данных в train файле: {train_count}')
print(f'Количество данных в val файле: {val_count}')
print(f'Количество данных в test файле: {test_count}')

# 2. Количество данных по лейблам

In [ ]:
train_category_count = count_category_data(train_filename)
val_category_count = count_category_data(val_filename)
test_category_count = count_category_data(test_filename)

In [ ]:
for i, data in enumerate([train_category_count, val_category_count, test_category_count]):

    if i == 0:
        k = "Train"
    elif i == 1:
        k = "Val"
    elif i == 2:
        k = "Test"

    is_up_false = data["is_up"]["false"]
    is_up_true = data["is_up"]["true"]

    color_White = data["color"]["White"]
    color_Yellow = data["color"]["Yellow"]
    color_Nocolor = data["color"]["No color"]

    count_undefined = data["count"]["undefined"]
    count_K1 = data["count"]["K1"]
    count_K2_1 = data["count"]["K2_1"]
    count_K2_2 = data["count"]["K2_2"]
    count_K3 = data["count"]["K3"]
    count_K4 = data["count"]["K4"]

    print(
        f"{k} - is_up: false: {is_up_false}, true: {is_up_true}\n"
        f"        color: White: {color_White}, Yellow: {color_Yellow}, No color: {color_Nocolor}\n"
        f"        count: undefined: {count_undefined}, K1: {count_K1}, K2_1: {count_K2_1}, K2_2: {count_K2_2}, K3: {count_K3}, K4: {count_K4}")

# 3. Аугментация

In [ ]:
from src.data.components.augmentation import get_transforms_from_config
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import os

In [ ]:
path_config_transform = "../configs/data/augmentation_train.yaml"
test_image = "../dataset_coco/images/1658918826546_00049000_2.jpg"
transforms=get_transforms_from_config(path_config_transform)

In [ ]:
image_pil = Image.open(test_image).convert("RGB")
image = np.array(image_pil)
print("Размер изображения до трансформации: ", image.shape)
#оригинальное изображение
image_pil.show()

In [ ]:
transform_image = transforms(image = image)
img_np = transform_image["image"]
print("Размер изображения после трансформации: ", img_np.shape)

In [ ]:
img_np = transform_image["image"]
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Обратное преобразование (inverse normalization)
img_np = std * img_np + mean
img_np = np.clip(img_np, 0, 1)

augmented_image_pil = Image.fromarray((img_np * 255).astype(np.uint8))
#изображение после аушментации
augmented_image_pil.show()